## Installs.

In [1]:
# installs.
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 


## Imports.

In [2]:
# imports.
import pandas as pd
import numpy as np
import requests
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
pd.set_option('display.float_format', lambda x: '%.2f' % x)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# read in the cleaned original data.
df_tr = pd.read_csv('https://raw.githubusercontent.com/CVanchieri/DataSets/master/WorldBankDeforestation/WorldBank_1990_2018.csv')

## Train Data.

In [4]:
# Set train data.
train = df_tr.copy()
train = train.drop(columns=['Unnamed: 0'])
train.head()

,Country Code,Year,Agricultural land (sq. km),GDP per capita growth (annual %),Livestock production index (2004-2006 = 100),Urban population,Crop production index (2004-2006 = 100),Food production index (2004-2006 = 100),Ores and metals exports (% of merchandise exports),Electric power consumption (kWh per capita),Forest area (% of land area)
0,ABW,1990,20.00,2.09,67.49,31273.00,71.69,69.36,1.10,1237.52,2.33
1,AFG,1990,380400.00,1.32,70.69,2628554.00,66.64,68.12,2.75,1237.52,2.07
2,AGO,1990,574040.00,-6.66,70.11,4400964.00,29.25,37.91,6.22,53.17,48.91
3,ALB,1990,11210.00,-11.19,57.97,1197222.00,84.36,68.73,2.75,552.25,28.79
4,AND,1990,230.00,-0.14,67.49,51627.00,71.69,69.36,2.75,1237.52,34.04


### Train/Val Split.

In [5]:
# Set variables.
features = train.columns[:-1].tolist()
target = 'Forest area (% of land area)'

X = train.drop(columns=target)
y = train[target]

In [6]:
# Use train/val split on the data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((4899, 10), (4899,), (1225, 10), (1225,), (1532, 10), (1532,))

### Random Forest Pipeline.

In [7]:
# random forest pipeline.
pipeline = make_pipeline(
    ce.OneHotEncoder(), 
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
)
pipeline.fit(X_train, y_train)

print ('Training Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))
y_pred = pipeline.predict(X_val)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Training Accuracy 0.9986954216649627
Validation Accuracy 0.9929648523153405


## Test Data.

In [8]:
# read in the cleaned predictions data frame.
df_te = pd.read_csv('https://raw.githubusercontent.com/CVanchieri/DataSets/master/WorldBankDeforestation/WorldBank_2019_2120.csv')

In [9]:
# Set the test data.
test = df_te.copy()
test = test.drop(columns=['Unnamed: 0'])
test.head()

,Country Code,Year,Agricultural land (sq. km),GDP per capita growth (annual %),Livestock production index (2004-2006 = 100),Urban population,Crop production index (2004-2006 = 100),Food production index (2004-2006 = 100),Ores and metals exports (% of merchandise exports),Electric power consumption (kWh per capita),Forest area (% of land area)
0,ABW,2019,20.00,-0.75,123.79,48057.97,123.08,125.53,6.37,2712.14,2.33
1,AFG,2019,378959.53,3.31,108.26,9262840.16,148.75,130.08,2.05,2712.14,2.07
2,AGO,2019,591467.93,2.39,146.48,19384636.41,226.44,208.66,2.28,308.43,46.05
3,ALB,2019,11904.29,6.65,120.60,1709176.32,178.59,151.17,12.88,2608.40,28.14
4,AND,2019,184.99,0.81,123.79,75938.01,123.08,125.53,3.59,2712.14,34.04


### Random Forest Pipeline.

In [10]:
# set the variables.
features = test.columns[:-1].tolist()
target = 'Forest area (% of land area)'

X_test = test[features]
y_test = test[target]

pipeline.fit(X_test, y_test)
y_pred = pipeline.predict(X_test)

print ('Test Accuracy', pipeline.score(X_test, y_test))
y_pred

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Test Accuracy 0.9998526451418099


array([  2.4116666 ,   2.2803321 ,  44.96266689, ...,   7.47180292,
        42.56087829, -49.36206047])

In [11]:
# Set the predictions on the data frame.
test['Forest area (% of land area)'] = pd.Series(y_pred)
# Merge the train and test data for one dataframe.
predictions = pd.concat([train, test])

In [12]:
# show the data frame shape.
print(predictions.shape)
# show the data frame with headers.
predictions.head()

(34584, 11)


,Country Code,Year,Agricultural land (sq. km),GDP per capita growth (annual %),Livestock production index (2004-2006 = 100),Urban population,Crop production index (2004-2006 = 100),Food production index (2004-2006 = 100),Ores and metals exports (% of merchandise exports),Electric power consumption (kWh per capita),Forest area (% of land area)
0,ABW,1990,20.00,2.09,67.49,31273.00,71.69,69.36,1.10,1237.52,2.33
1,AFG,1990,380400.00,1.32,70.69,2628554.00,66.64,68.12,2.75,1237.52,2.07
2,AGO,1990,574040.00,-6.66,70.11,4400964.00,29.25,37.91,6.22,53.17,48.91
3,ALB,1990,11210.00,-11.19,57.97,1197222.00,84.36,68.73,2.75,552.25,28.79
4,AND,1990,230.00,-0.14,67.49,51627.00,71.69,69.36,2.75,1237.52,34.04


In [13]:
# Function to add in some country names.
def label_race (row):
   if row['Country Code'] == 'USA' :
      return 'United States of America'
   if row['Country Code'] == 'CHA' :
      return 'China'
   if row['Country Code'] == 'CAN' :
      return 'Canada'
   if row['Country Code'] == 'AUS' :
      return 'Australia'
   if row['Country Code'] == 'ARG' :
      return 'Argentina'
   if row['Country Code'] == 'BRA':
      return 'Brazil'
   if row['Country Code'] == 'BEL':
      return 'Belgium'   
   if row['Country Code'] == 'CHL':
      return 'Chile'
   if row['Country Code'] == 'DEU' :
      return 'Germany'
   if row['Country Code'] == 'ZAF' :
      return 'South Africa'
   if row['Country Code']  == 'NZL':
      return 'New Zealnd'
   if row['Country Code'] == 'GBR':
      return 'United Kingdom'
   if row['Country Code'] == 'IND' :
      return 'India'
   if row['Country Code'] == 'KHM' :
      return 'Cambodia'
   if row['Country Code']  == 'THA':
      return 'Thailand'
   if row['Country Code'] == 'VNM':
      return 'Vietnam'
   if row['Country Code'] == 'HIC' :
      return 'High Income Countries'
   if row['Country Code']  == 'MIC':
      return 'Middle Income Countries'
   if row['Country Code'] == 'LIC':
      return 'Low Income Countries'
   return 'Other'

In [14]:
# make a copy of the dataframe.
final_c = predictions.copy()

In [15]:
# Apply the function to the datarame.
final_c.apply (lambda row: label_race(row), axis=1)
final_c['Country Name'] = final_c.apply (lambda row: label_race(row), axis=1)
final_c = final_c[['Country Name',
                                  'Country Code',
                                  'Year',
                                  'Agricultural land (sq. km)',
                                  'Electric power consumption (kWh per capita)',
                                  'GDP per capita growth (annual %)',
                                  'Livestock production index (2004-2006 = 100)',
                                  'Ores and metals exports (% of merchandise exports)',
                                  'Urban population',
                                  'Crop production index (2004-2006 = 100)',
                                  'Food production index (2004-2006 = 100)',
                                  'Forest area (% of land area)']]

In [16]:
# Clean the dataframe.
final_c = final_c[final_c['Country Name'] != 'Other']
final_c = final_c[['Country Name',
 'Country Code',
 'Year',
 'Agricultural land (sq. km)',
 'Electric power consumption (kWh per capita)',
 'GDP per capita growth (annual %)',
 'Livestock production index (2004-2006 = 100)',
 'Ores and metals exports (% of merchandise exports)',
 'Urban population',
 'Crop production index (2004-2006 = 100)',
 'Food production index (2004-2006 = 100)',
 'Forest area (% of land area)']]
final_c['Forest area (% of land area)'] = final_c['Forest area (% of land area)'].clip(lower=0)

In [17]:
print(final_c.shape)
final_c.tail()

(2358, 12)


,Country Name,Country Code,Year,Agricultural land (sq. km),Electric power consumption (kWh per capita),GDP per capita growth (annual %),Livestock production index (2004-2006 = 100),Ores and metals exports (% of merchandise exports),Urban population,Crop production index (2004-2006 = 100),Food production index (2004-2006 = 100),Forest area (% of land area)
26842,New Zealnd,NZL,2120,-135584.43,10434.64,2.23,300.90,-3.78,8517411.94,299.57,322.48,44.84
26895,Thailand,THA,2120,252837.49,9786.31,-9.02,310.28,4.64,107322891.44,375.28,334.88,44.16
26913,United States of America,USA,2120,3297611.47,16616.36,-0.82,226.75,8.22,563473525.03,265.25,283.85,38.27
26919,Vietnam,VNM,2120,340196.12,7256.84,3.86,670.84,-2.85,106616457.51,512.49,513.35,127.97
26925,South Africa,ZAF,2120,960131.69,4663.79,7.44,391.42,130.12,105234320.88,246.60,323.85,7.47


In [18]:
c = final_c[final_c['Country Name'] == 'Cambodia']

In [19]:
c.tail()

,Country Name,Country Code,Year,Agricultural land (sq. km),Electric power consumption (kWh per capita),GDP per capita growth (annual %),Livestock production index (2004-2006 = 100),Ores and metals exports (% of merchandise exports),Urban population,Crop production index (2004-2006 = 100),Food production index (2004-2006 = 100),Forest area (% of land area)
25729,Cambodia,KHM,2116,100646.21,-2675.94,41.02,214.12,-5.89,11427803.95,903.85,772.92,0.00
25993,Cambodia,KHM,2117,101093.97,-2702.71,41.35,215.26,-5.96,11507286.54,910.90,778.86,0.00
26257,Cambodia,KHM,2118,101541.72,-2729.48,41.69,216.41,-6.02,11586769.12,917.95,784.79,0.00
26521,Cambodia,KHM,2119,101989.48,-2756.24,42.02,217.56,-6.09,11666251.70,924.99,790.73,0.00
26785,Cambodia,KHM,2120,102437.24,-2783.01,42.36,218.71,-6.15,11745734.29,932.04,796.66,0.00


In [20]:
"""# Download predictions result as a csv.
from google.colab import files 
final_c.to_csv('Deforestation_Predictions.csv', index=False)
files.download('Deforestation_Predictions.csv')"""

"# Download predictions result as a csv.\nfrom google.colab import files \nfinal_c.to_csv('Deforestation_Predictions.csv', index=False)\nfiles.download('Deforestation_Predictions.csv')"

In [21]:
"""# Download predictions result as a json.
final_c.to_json('Deforestation_Predictions.json', orient= 'records')
files.download('Deforestation_Predictions.json')"""

"# Download predictions result as a json.\nfinal_c.to_json('Deforestation_Predictions.json', orient= 'records')\nfiles.download('Deforestation_Predictions.json')"